**Feladat szövege**


---


A cél egy olyan hangalapú chatbot létrehozása, amellyel időpontot lehet foglalni egy szolgáltatóhoz. A párbeszédben a legfontosabb információkat kell megkérdezni a megerősítéshez. A felhasználó azonosítása (név, e-mail, telefonszám vagy más módszer), időpont, választott kezelés vagy szolgáltatás, egyéb kért extra információk (pl. érzelmi állapot). Az eredményeknek json formátumban kell rendelkezésre állniuk. A rendszernek angol és/vagy magyar nyelven kell működnie.

https://huggingface.co/microsoft/Phi-4-mini-instruct

https://github.com/rhasspy/piper

https://github.com/openai/whisper

In [1]:
!pip install --quiet transformers accelerate torch gradio faiss-cpu sentence-transformers openai-whisper piper-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr
import whisper
from datasets import load_dataset
import soundfile as sf
import time

import tempfile
import os
import scipy.io.wavfile
import requests
import subprocess
import re

In [3]:
# 3. GPU detektálása
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [4]:
#STT/ASR (whisper) modell kiválasztása
#whisper_model = whisper.load_model("base")  # vagy "small", "medium", "large"
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:14<00:00, 9.84MiB/s]


In [5]:
#TTS with Piper - switch to CLI invocation
def synthesize_speech_piper(text, output_path="tts_output.wav", lang="hu"):
    """
    Szöveg -> hangfájl (wav) (Piper TTS via CLI)
    """
    # Choose appropriate Piper voice model
    model_id = "hu_HU-anna-medium" if lang == "hu" else "en_US-joe-medium"
    # Build CLI command
    cmd = [
        "piper",
        "--model", model_id,
        "--output_file", output_path
    ]
    # Invoke Piper CLI, feeding text via stdin
    subprocess.run(cmd, input=text.encode("utf-8"), check=True)
    return output_path

In [6]:
# --- STT: Audio (.wav) → Text ---
def transcribe_audio_whisper(audio_path, lang="hu"):
    """
    Megadott wav formátumú hangfájl → szöveg Whisper ASR-rel.
    """
    result = whisper_model.transcribe(audio_path, language=lang)
    return result["text"]

In [7]:
# 4. Tudásbázis: Időpontok és szolgáltatások
# Példák időpontokra és szolgáltatásokra
# Ezeket a chatbot a RAG során használja kontextusként
documents = [
    "Elérhető szolgáltatások: hajvágás, masszázs, arckezelés, manikűr, pedikűr.",
    "Időpontfoglalás hétfőtől péntekig 8:00 és 18:00 között lehetséges.",
    "A szolgáltatások időtartama: hajvágás 30 perc, masszázs 60 perc, arckezelés 45 perc, manikűr 40 perc, pedikűr 50 perc.",
    "Foglaláshoz szükséges adatok: név, e-mail cím vagy telefonszám, választott szolgáltatás, kívánt időpont.",
    "Lemondás vagy módosítás esetén kérjük, legalább 24 órával előbb jelezze!"
]

In [8]:
# 5. Embedding modell és indexelés
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# 6. LLM betöltése (Phi-4-mini-instruct)
model_id = "microsoft/phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [10]:
# Slot-filling + NER: foglalási adatok kinyerése LLM-mel
def extract_slots_ner(user_utterance, lang="hu"):
    """
    Slot-filling + NER: kinyeri a foglaláshoz szükséges adatokat a szövegből.
    Visszaadott dict: {"name":..., "email":..., "phone":..., "datetime":..., "service":...}
    """
    # LLM-alapú prompt a főbb entitásokra
    if lang == "hu":
        prompt = (
            "Az alábbi mondatból nyerd ki a következő adatokat, ha szerepelnek: név, e-mail, telefonszám, időpont, szolgáltatás. "
            "A válasz legyen JSON formátumú, pl.: {\"name\":..., \"email\":..., \"phone\":..., \"datetime\":..., \"service\":...}.\n"
            f"Mondat: {user_utterance}\nJSON:"
        )
    else:
        prompt = (
            "From the following sentence, extract the following fields if present: name, email, phone, datetime, service. "
            "Return as JSON, e.g.: {\"name\":..., \"email\":..., \"phone\":..., \"datetime\":..., \"service\":...}.\n"
            f"Sentence: {user_utterance}\nJSON:"
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=80)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # JSON kinyerése a válaszból
    match = re.search(r'\{.*\}', answer, re.DOTALL)
    if match:
        import json
        try:
            slots = json.loads(match.group(0))
        except Exception:
            slots = {}
    else:
        slots = {}
    return slots

# Slotok listája
REQUIRED_SLOTS = ["name", "email", "phone", "datetime", "service"]

# Slot-filling állapotkezelés a párbeszéd során
# Egyszerűsített: minden körben frissítjük a slotokat, és csak a hiányzókat kérdezzük

def update_slots(current_slots, new_slots):
    """
    Frissíti a slotokat: ha új érték érkezik, felülírja a régit.
    """
    for k, v in new_slots.items():
        if v and (k not in current_slots or not current_slots[k]):
            current_slots[k] = v
    return current_slots

def get_missing_slots(slots):
    """
    Visszaadja a hiányzó slotok listáját.
    """
    return [slot for slot in REQUIRED_SLOTS if slot not in slots or not slots[slot]]

In [19]:
# Egyszerű érzelemfelismerő: alapérzelmek visszaadása (öröm, szomorúság, harag, félelem, meglepetés, undor, semleges)
import re

def detect_emotion_llm(user_utterance, lang="hu"):
    """
    Alapérzelmek detektálása kulcsszavak alapján.
    Magyar: 'öröm', 'szomorúság', 'harag', 'félelem', 'meglepetés', 'undor', 'semleges'
    Angol: 'joy', 'sadness', 'anger', 'fear', 'surprise', 'disgust', 'neutral'
    """
    emotions_hu = {
        'öröm': [r'öröm', r'boldog', r'vidám', r'elégedett', r'mosoly', r'köszönöm', r'köszi'],
        'szomorúság': [r'szomorú', r'bánat', r'sajnál', r'sír', r'csalódott'],
        'harag': [r'harag', r'düh', r'mérges', r'ideges', r'bosszús'],
        'félelem': [r'félelem', r'félek', r'aggód', r'ijed', r'paráz'],
        'meglepetés': [r'meglep', r'váratlan', r'azt hittem', r'nem gondoltam'],
        'undor': [r'undor', r'utál', r'rossz', r'kellemetlen'],
    }
    emotions_en = {
        'joy': [r'joy', r'happy', r'glad', r'smile', r'thank'],
        'sadness': [r'sad', r'sorrow', r'sorry', r'cry', r'disappointed'],
        'anger': [r'angry', r'mad', r'annoyed', r'furious', r'upset'],
        'fear': [r'fear', r'afraid', r'scared', r'worry', r'anxious'],
        'surprise': [r'surprise', r'surprised', r'unexpected', r'didn\'t expect'],
        'disgust': [r'disgust', r'hate', r'awful', r'unpleasant'],
    }
    text = user_utterance.lower()
    if lang == "hu":
        for emotion, patterns in emotions_hu.items():
            for pat in patterns:
                if re.search(pat, text):
                    return emotion
        return "semleges"
    else:
        for emotion, patterns in emotions_en.items():
            for pat in patterns:
                if re.search(pat, text):
                    return emotion
        return "neutral"

In [11]:
#Zero Shot Intent Detection
def detect_intent_llm(user_utterance, lang="hu"):
    """
    Zero-shot intent detection LLM-mel.
    user_utterance: a felhasználó mondata
    lang: 'hu' vagy 'en'
    Visszaadott intent: szöveg (pl. 'foglalás', 'info', stb.)
    """
    if lang == "hu":
        prompt = (
            "Az alábbi mondat alapján válaszd ki a szándékot az alábbiak közül (csak egy szót adj vissza, szóköz nélkül): "
            "foglalás, lemondás, módosítás, információ, köszönés, egyéb.\n"
            f"Felhasználó: {user_utterance}\nSzándék:"
        )
    else:
        prompt = (
            "Based on the following sentence, choose the intent from: booking, cancel, reschedule, info, greeting, other. "
            "Return only one word, no spaces.\n"
            f"User: {user_utterance}\nIntent:"
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=3)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Szándék kiszűrése a válaszból
    intent = answer.split(":")[-1].strip().split()[0]
    return intent

In [12]:
def rag_phi4mini_chatbot(user_input, input_mode="text", lang="hu"):
    """
    Chatbot, ami támogatja a hang- és szövegalapú chatet is.
    user_input: szöveg (input_mode="text") vagy hangfájl útvonal (input_mode="audio")
    input_mode: 'text' vagy 'audio'
    lang: 'hu' vagy 'en'
    return: (válasz_szöveg, intent, emotion, válasz_hangfájl_útvonal)
    """
    # 1. Ha hang, futtasd le a STT-t
    if input_mode == "audio":
        user_utterance = transcribe_audio_whisper(user_input, lang=lang)
    else:
        user_utterance = user_input

    # 2. Intent detektálás
    intent = detect_intent_llm(user_utterance, lang=lang)

    # 3. Érzelem detektálás
    emotion = detect_emotion_llm(user_utterance, lang=lang)

    # 4. Embed és keresés (RAG)
    q_embed = embedder.encode([user_utterance])
    D, I = index.search(np.array(q_embed), k=1)
    context = documents[I[0][0]]

    # Role definition for system message
    role_descr_hu = (
        "Te egy barátságos időpontfoglaló asszisztens vagy. Ha hiányzó információk vannak, kérdezz rájuk, "
        "válaszod legyen természetes nyelvű és közvetlen, ne JSON-ban add vissza."
    )
    role_descr_en = (
        "You are a friendly appointment-booking assistant. Ask for any missing details, "
        "and respond conversationally in natural language. Do not return JSON."
    )
    # 5. Prompt építés (base prompt)
    if D[0][0] > 1.0:
        base_prompt = f"Kérdés: {user_utterance}\nSzándék: {intent}\nVálasz:"
    else:
        base_prompt = f"""
Az alábbi kontextus alapján válaszolj a kérdésre magyarul:

KONTEKSTUS:
{context}

KÉRDÉS:
{user_utterance}

SZÁNDÉK:
{intent}

VÁLASZ:
"""
    # Wrap with system role tokens
    prompt = (
        f"<|system|>{role_descr_hu if lang=='hu' else role_descr_en}<|end|>"
        f"{base_prompt}"
    )
    # 6. LLM generálás
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "VÁLASZ:" in answer:
        answer = answer.split("VÁLASZ:")[-1].strip()
    elif "Válasz:" in answer:
        answer = answer.split("Válasz:")[-1].strip()

    # 7. TTS – szövegből hangfájlt generálunk
    tts_output_path = synthesize_speech_piper(answer, output_path="tts_output.wav", lang=lang)

    return answer, intent, emotion, tts_output_path

In [13]:
def gradio_chat_interface(text_input, audio_input, input_mode, lang, slot_state):
    try:
        if input_mode == "text":
            if not text_input or text_input.strip() == "":
                return "Adj meg kérdést!", "", "", slot_state, None
            answer, intent, emotion, slot_state, wav_path = rag_phi4mini_chatbot(text_input, input_mode="text", lang=lang, slot_state=slot_state)
        else:
            if audio_input is None:
                return "Nincs hang bemenet.", "", "", slot_state, None
            sr, data = audio_input
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                scipy.io.wavfile.write(tmp.name, sr, data)
                answer, intent, emotion, slot_state, wav_path = rag_phi4mini_chatbot(tmp.name, input_mode="audio", lang=lang, slot_state=slot_state)
            os.remove(tmp.name)

        print("TTS path:", wav_path)
        print("Fájl létezik:", os.path.exists(wav_path))
        if not wav_path or not os.path.exists(wav_path):
            return answer, intent, emotion, slot_state, None
        return answer, intent, emotion, slot_state, wav_path

    except Exception as e:
        import traceback
        hiba = traceback.format_exc()
        return f"HIBA: {hiba}", "", "", slot_state, None

In [20]:
# Gradio hibakezelés: hibaüzenet megjelenítése textboxban
import gradio as gr

def format_error_message(msg):
    if msg and (msg.startswith("HIBA:") or msg.startswith("ERROR:")):
        return msg  # sima szöveg, nincs HTML formázás
    return msg

with gr.Blocks() as demo:
    gr.Markdown("### RAG Chatbot - Microsoft Phi-4-mini-instruct + Whisper STT + Piper TTS")
    gr.Markdown("Kérdezz szöveggel vagy szóban! A chatbot a kontextus alapján válaszol magyarul (és hangban is).\nA foglalási adatok automatikusan kitöltődnek a párbeszéd során.")
    with gr.Row():
        input_mode = gr.Radio(["text", "audio"], value="text", label="Bemenet típusa")
        lang = gr.Radio(["hu", "en"], value="hu", label="Nyelv")
    with gr.Row():
        text_input = gr.Textbox(lines=2, label="Kérdés (ha szöveges)")
        audio_input = gr.Audio(type="numpy", label="Kérdés (ha hang)")
    submit_btn = gr.Button("Küldés")
    output_text = gr.Textbox(label="Chatbot válasza")
    output_intent = gr.Textbox(label="Felismer szándék (intent)")
    output_emotion = gr.Textbox(label="Felismer érzelem (emotion)")
    output_slots = gr.JSON(label="Foglalási adatok (slotok)")
    output_audio = gr.Audio(label="Chatbot válasza (hang)", autoplay=True)
    slot_state = gr.State({})

    def gradio_chat_interface_textbox(text_input, audio_input, input_mode, lang, slot_state):
        try:
            if input_mode == "text":
                if not text_input or text_input.strip() == "":
                    return format_error_message("Adj meg kérdést!"), "", "", {}, None, slot_state
                answer, intent, emotion, wav_path = rag_phi4mini_chatbot(text_input, input_mode="text", lang=lang)
            else:
                if audio_input is None:
                    return format_error_message("Nincs hang bemenet."), "", "", {}, None, slot_state
                sr, data = audio_input
                with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                    scipy.io.wavfile.write(tmp.name, sr, data)
                    answer, intent, emotion, wav_path = rag_phi4mini_chatbot(tmp.name, input_mode="audio", lang=lang)
                os.remove(tmp.name)

            if not wav_path or not os.path.exists(wav_path):
                return format_error_message(answer), intent, emotion, {}, None, slot_state
            return format_error_message(answer), intent, emotion, {}, wav_path, slot_state
        except Exception as e:
            import traceback
            hiba = traceback.format_exc()
            return format_error_message(f"HIBA: {hiba}"), "", "", {}, None, slot_state

    submit_btn.click(
        gradio_chat_interface_textbox,
        inputs=[text_input, audio_input, input_mode, lang, slot_state],
        outputs=[output_text, output_intent, output_emotion, output_slots, output_audio, slot_state]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d4d9238694bfb9d70.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
